In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

#https://www.tensorflow.org/datasets/keras_example

/Users/mariiakokina/Downloads/py39env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


### Load a dataset

In [2]:
# Download dataset
(ds_train, ds_val, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train[:90%]', 'train[90%:]', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

### Build a training and an evaluation pipeline

In [3]:
# Helper function to convert datasets to NumPy arrays
def ds_to_numpy(ds):
    images = []
    labels = []
    for img, label in tfds.as_numpy(ds):
        images.append(img)
        labels.append(label)
    return np.array(images), np.array(labels)

# Convert datasets
X_train, y_train = ds_to_numpy(ds_train)
X_val, y_val = ds_to_numpy(ds_val)
X_test, y_test = ds_to_numpy(ds_test)

# Normalize the data (important!)
X_train = X_train / 255.0
X_val = X_val / 255.0
X_test = X_test / 255.0

### Create and train the model

In [4]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Reshape((28, 28, 1), input_shape=(28, 28)),  # ensure channel dimension
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)  # from_logits = True in your loss
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    X_train, 
    y_train, 
    epochs=10,
    validation_data=(X_val, y_val)
)

Epoch 1/10


/Users/mariiakokina/Downloads/py39env/lib/python3.9/site-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1688/1688 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - loss: 0.2983 - sparse_categorical_accuracy: 0.9065 - val_loss: 0.0502 - val_sparse_categorical_accuracy: 0.9857
Epoch 2/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - loss: 0.0444 - sparse_categorical_accuracy: 0.9866 - val_loss: 0.0399 - val_sparse_categorical_accuracy: 0.9872
Epoch 3/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 13s 8ms/step - loss: 0.0274 - sparse_categorical_accuracy: 0.9914 - val_loss: 0.0423 - val_sparse_categorical_accuracy: 0.9882
Epoch 4/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 14s 8ms/step - loss: 0.0194 - sparse_categorical_accuracy: 0.9941 - val_loss: 0.0383 - val_sparse_categorical_accuracy: 0.9885
Epoch 5/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 14s 8ms/step - loss: 0.0121 - sparse_categorical_accuracy: 0.9965 - val_loss: 0.0393 - val_sparse_categorical_accuracy: 0.9903
Epoch 6/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 14s 8ms/step - loss: 0.0115 - sparse_categorical_accuracy: 0.9960 - val_loss: 0.0381 - val_sparse_categorical_accuracy: 0.99

In [5]:
# Evaluate on val set
train_loss, train_acc = model.evaluate(X_val, y_val, verbose=1)

188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0511 - sparse_categorical_accuracy: 0.9903


In [6]:
# Required imports
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
from time import time

# Define CNN model creation function compatible with scikeras wrapper
def create_cnn_model(hidden_layer_sizes=(64,), learning_rate=0.001):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        Flatten(),
        Dense(hidden_layer_sizes[0], activation='relu'),
        Dense(10, activation='softmax')
    ])

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Function to display Grid Search results clearly
def display_cv_results(search_results):
    print('Best score = {:.4f} using {}'.format(search_results.best_score_, search_results.best_params_))
    means = search_results.cv_results_['mean_test_score']
    stds = search_results.cv_results_['std_test_score']
    params = search_results.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print('Mean accuracy +/- std = {:.4f} +/- {:.4f} with: {}'.format(mean, stdev, param))

# Creating KerasClassifier wrapper for scikeras
model = KerasClassifier(model=create_cnn_model, verbose=1)

# Define parameter grid for GridSearchCV
param_grid = {
    'batch_size': [32, 64],
    'epochs': [10, 20, 30],
    'model__hidden_layer_sizes': [(32,), (64,), (128,)],
    'model__learning_rate': [0.001, 0.01]
}

# Start timing
start = time()

# Setting up GridSearchCV with scikeras

grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, verbose=2, n_jobs=-1)

# Fit GridSearchCV to your data
grid_result = grid.fit(X_train, y_train)

# Display execution time
print('Time for grid search = {:.0f} sec'.format(time()-start))

# Display results
print("\nGrid search results:")
display_cv_results(grid_result)


Fitting 3 folds for each of 36 candidates, totalling 108 fits


/Users/mariiakokina/Downloads/py39env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/mariiakokina/Downloads/py39env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/mariiakokina/Downloads/py39env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/mariiakokina/Downloads/py39env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, current

Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 32s 28ms/step - accuracy: 0.8661 - loss: 0.3983
Epoch 2/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 32s 27ms/step - accuracy: 0.8726 - loss: 0.3830
Epoch 2/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 33s 28ms/step - accuracy: 0.8723 - loss: 0.3954
Epoch 2/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 33s 28ms/step - accuracy: 0.8466 - loss: 0.4932
Epoch 2/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 33s 28ms/step - accuracy: 0.8691 - loss: 0.4434
  27/1125 ━━━━━━━━━━━━━━━━━━━━ 31s 29ms/step - accuracy: 0.9872 - loss: 0.0639Epoch 2/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 33s 28ms/step - accuracy: 0.8470 - loss: 0.4738
Epoch 2/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 34s 29ms/step - accuracy: 0.8598 - loss: 0.4593
Epoch 2/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 34s 29ms/step - accuracy: 0.8512 - loss: 0.4894
Epoch 2/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 32s 29ms/step - accuracy: 0.9703 - loss: 0.1009
Epoch 3/10
1125/1125 ━━━━━━━━━━━━━━

/Users/mariiakokina/Downloads/py39env/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


 662/1125 ━━━━━━━━━━━━━━━━━━━━ 14s 32ms/step - accuracy: 0.9970 - loss: 0.0092Epoch 1/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 8s 14ms/stepep - accuracy: 0.9959 - loss: 0.01195
563/563 ━━━━━━━━━━━━━━━━━━━━ 8s 14ms/stepep - accuracy: 0.9959 - loss: 0.0119
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.9967 - loss: 0.0096[CV] END batch_size=32, epochs=10, model__hidden_layer_sizes=(32,), model__learning_rate=0.01; total time= 5.9min
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 37s 33ms/step - accuracy: 0.9967 - loss: 0.0096
1111/1125 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.9959 - loss: 0.0113[CV] END batch_size=32, epochs=10, model__hidden_layer_sizes=(32,), model__learning_rate=0.01; total time= 5.9min
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 37s 32ms/step - accuracy: 0.9959 - loss: 0.0113
 21/563 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step

...

 19/563 ━━━━━━━━━━━━━━━━━━━━ 28s 52ms/step - accuracy: 0.9980 - loss: 0.0038Epoch 1/30
563/563 ━━━━━━━━━━━━━━━━━━━━ 30s 52ms/step - accuracy: 0.9812 - loss: 0.0839-0
Epoch 15/30
563/563 ━━━━━━━━━━━━━━━━━━━━ 30s 53ms/step - accuracy: 0.9998 - loss: 9.1088e-04
Epoch 29/30
563/563 ━━━━━━━━━━━━━━━━━━━━ 28s 49ms/step - accuracy: 0.9870 - loss: 0.0705
563/563 ━━━━━━━━━━━━━━━━━━━━ 28s 49ms/step - accuracy: 0.9872 - loss: 0.0759
Epoch 30/30
282/282 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/stepp - accuracy: 0.7529 - loss: 0.706
346/563 ━━━━━━━━━━━━━━━━━━━━ 10s 48ms/step - accuracy: 0.9985 - loss: 0.0042[CV] END batch_size=64, epochs=30, model__hidden_layer_sizes=(64,), model__learning_rate=0.01; total time=14.8min
350/563 ━━━━━━━━━━━━━━━━━━━━ 10s 49ms/step - accuracy: 0.9985 - loss: 0.0042

/Users/mariiakokina/Downloads/py39env/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


359/563 ━━━━━━━━━━━━━━━━━━━━ 9s 48ms/step - accuracy: 0.9878 - loss: 0.0595Epoch 1/30
228/563 ━━━━━━━━━━━━━━━━━━━━ 16s 48ms/step - accuracy: 0.9867 - loss: 0.0622[CV] END batch_size=64, epochs=30, model__hidden_layer_sizes=(128,), model__learning_rate=0.01; total time=   6.1s
563/563 ━━━━━━━━━━━━━━━━━━━━ 28s 50ms/step - accuracy: 0.9980 - loss: 0.0056
Epoch 16/30
563/563 ━━━━━━━━━━━━━━━━━━━━ 28s 49ms/step - accuracy: 0.9875 - loss: 0.0616
563/563 ━━━━━━━━━━━━━━━━━━━━ 28s 50ms/step - accuracy: 0.9988 - loss: 0.0037
Epoch 30/30
563/563 ━━━━━━━━━━━━━━━━━━━━ 27s 48ms/step - accuracy: 0.9862 - loss: 0.0596
Epoch 16/30
563/563 ━━━━━━━━━━━━━━━━━━━━ 30s 47ms/step - accuracy: 0.8436 - loss: 0.4565
Epoch 2/30
282/282 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.9875 - loss: 0.0599
373/563 ━━━━━━━━━━━━━━━━━━━━ 9s 47ms/step - accuracy: 0.9989 - loss: 0.0030[CV] END batch_size=64, epochs=30, model__hidden_layer_sizes=(64,), model__learning_rate=0.01; total time=15.2min
563/563 ━━━━━━━━━━━━━━━━━━

/Users/mariiakokina/Downloads/py39env/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
1 fits failed out of a total of 108.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/mariiakokina/Downloads/py39env/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/mariiakokina/Downloads/py39env/lib/python3.9/site-packages/scikeras/wrappers.py", line 1501, in fit
    super().fit(X=X, y=y, sample_weight=sample_weight, **kwargs)
  File "/Users/mariiakokina/Downloads/py39env/lib/python3.9/site-packages/scikeras/wr

Epoch 1/30
844/844 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.8026 - loss: 0.5832
Epoch 2/30
844/844 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - accuracy: 0.9787 - loss: 0.0691
Epoch 3/30
844/844 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - accuracy: 0.9850 - loss: 0.0461
Epoch 4/30
844/844 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.9885 - loss: 0.0359
Epoch 5/30
844/844 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - accuracy: 0.9912 - loss: 0.0295
Epoch 6/30
844/844 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - accuracy: 0.9925 - loss: 0.0226
Epoch 7/30
844/844 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - accuracy: 0.9943 - loss: 0.0173
Epoch 8/30
844/844 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - accuracy: 0.9954 - loss: 0.0149
Epoch 9/30
844/844 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.9963 - loss: 0.0119
Epoch 10/30
844/844 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - accuracy: 0.9964 - loss: 0.0115
Epoch 11/30
844/844 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.9966 - loss: 0.0098
Epoch 12/30
844/844 ━━━━━━━━━━━━━━━━

In [11]:
# Display results
print("\nGrid search results:")
display_cv_results(grid_result)

# Evaluate the best model on the validation set
best_model = grid_result.best_estimator_
val_accuracy = best_model.score(X_val, y_val)

print(f'\nAccuracy of the best model on validation set: {val_accuracy:.4f}')


Grid search results:
Best score = 0.9910 using {'batch_size': 64, 'epochs': 30, 'model__hidden_layer_sizes': (32,), 'model__learning_rate': 0.001}
Mean accuracy +/- std = 0.9871 +/- 0.0024 with: {'batch_size': 32, 'epochs': 10, 'model__hidden_layer_sizes': (32,), 'model__learning_rate': 0.001}
Mean accuracy +/- std = 0.9710 +/- 0.0039 with: {'batch_size': 32, 'epochs': 10, 'model__hidden_layer_sizes': (32,), 'model__learning_rate': 0.01}
Mean accuracy +/- std = 0.9867 +/- 0.0018 with: {'batch_size': 32, 'epochs': 10, 'model__hidden_layer_sizes': (64,), 'model__learning_rate': 0.001}
Mean accuracy +/- std = 0.9719 +/- 0.0022 with: {'batch_size': 32, 'epochs': 10, 'model__hidden_layer_sizes': (64,), 'model__learning_rate': 0.01}
Mean accuracy +/- std = 0.9888 +/- 0.0008 with: {'batch_size': 32, 'epochs': 10, 'model__hidden_layer_sizes': (128,), 'model__learning_rate': 0.001}
Mean accuracy +/- std = 0.9730 +/- 0.0022 with: {'batch_size': 32, 'epochs': 10, 'model__hidden_layer_sizes': (12

In [12]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Reshape((28, 28, 1), input_shape=(28, 28)),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu'),  # Using the best hidden_layer_size (32,)
    tf.keras.layers.Dense(10) 
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),  # Using the best learning_rate 0.001
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)
# Concatenate training and validation sets
X_combined = np.concatenate((X_train, X_val), axis=0)
y_combined = np.concatenate((y_train, y_val), axis=0)

model.fit(
    X_combined, 
    y_combined, 
    epochs=30,  # Using the best epochs 30
    batch_size=32,  # Using the best batch_size 32
)

/Users/mariiakokina/Downloads/py39env/lib/python3.9/site-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - loss: 0.3638 - sparse_categorical_accuracy: 0.8894
Epoch 2/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - loss: 0.0540 - sparse_categorical_accuracy: 0.9833
Epoch 3/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - loss: 0.0383 - sparse_categorical_accuracy: 0.9882
Epoch 4/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - loss: 0.0271 - sparse_categorical_accuracy: 0.9914
Epoch 5/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - loss: 0.0193 - sparse_categorical_accuracy: 0.9939
Epoch 6/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - loss: 0.0145 - sparse_categorical_accuracy: 0.9957
Epoch 7/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - loss: 0.0131 - sparse_categorical_accuracy: 0.9961
Epoch 8/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - loss: 0.0102 - sparse_categorical_accuracy: 0.9973
Epoch 9/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - loss: 0.0076 - sparse_categorical_accuracy: 0.9979
Epoch 10/30
1875/1875 ━━━━━━

In [13]:
# Evaluate the final model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)

print(f'\nAccuracy of the final model on test set: {test_accuracy:.4f}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0608 - sparse_categorical_accuracy: 0.9918

Accuracy of the final model on test set: 0.9907
